# Configuración inicial y librerías

Se importan las librerías necesarias para llevar a cabo el preprocesamiento del dataset. Este conjunto de datos requiere transformación de fechas, codificación de variables categóricas, creación de nuevas características y manejo de valores faltantes. A continuación, se explican brevemente las librerías utilizadas:

- `polars`: manipulación de datos estructurados, se emplea polars en lugar de pandas para optimizar el uso de recursos computacionales.
- `numpy`: operaciones numéricas y matemáticas.
- `scipy.stats`: operaciones estadísticas como la moda.
- `sklearn.preprocessing.LabelEncoder`: codificación de variables categóricas.
- `matplotlib.pyplot`: visualización básica (opcional).
- `warnings`: suprime advertencias innecesarias en la ejecución.
- `gc` (garbage collector): optimiza el uso de memoria.



In [32]:
# Librerías
import polars as pd
import numpy as np
import scipy.stats
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import warnings
import gc

# Ignorar advertencias

warnings.filterwarnings("ignore")

# Activar garbage collector para habilitar la recoleccion de basura, esto mejora el rendimiento en ambientes 
# que tienen un uso intensivo de memoria como lo es el caso del manejo de los datasets de la actividad desarrollada

gc.enable()


# Pre-preprocesamiento de datos

### Convertir unidades (dias)
La función `convertir_dias(data, columnas, divisor=12, redondear=True, reemplazar=False)` se utiliza para transformar columnas que contienen valores de tiempo expresados en días negativos, como por ejemplo DAYS_BIRTH o DAYS_EMPLOYED se pasa de dias negativos a meses.

El parámetro columnas especifica la lista de variables que se desean transformar. El parámetro `divisor` determina en qué unidad se hará la conversión (en este caso 12, para convertirlo a meses).

Dentro del bucle, se invierte el signo del valor (porque los días vienen negativos en el dataset), se divide por el `divisor`. Luego, se eliminan los valores negativos resultantes, ya que estos suelen ser errores de codificación y se reemplazan por None.

In [33]:

def convertir_dias(data, columnas, divisor=12, redondear=True, reemplazar=False):
    for col in columnas:
        nueva_columna = "CONVERTIDO_" + str(col)
        valores_convertidos = -data[col] / divisor
        if redondear:
            valores_convertidos = valores_convertidos.round()
        valores_convertidos[valores_convertidos < 0] = None
        if reemplazar:
            data[col] = valores_convertidos
        else:
            data[nueva_columna] = valores_convertidos
    return data

### Crear variables logarítmicas
La función `crear_logaritmos(data, columnas, reemplazar=False)` se utiliza para aplicar una transformación logarítmica a las variables numéricas del dataset. Este tipo de transformación es común cuando se trabaja con variables como ingresos, montos de crédito, etc. Esto se debe a que las transformaciones logarítmicas pueden ayudar a corregir la asimetría de variables y mejorar la linealidad en modelos estadísticos.

In [34]:
def crear_logaritmos(data, columnas, reemplazar=False):
    for col in columnas:
        valores_log = np.log(data[col].abs() + 1)
        if reemplazar:
            data[col] = valores_log
        else:
            data["LOG_" + str(col)] = valores_log
    return data


### Codificar variables categóricas
Mucha de la información que se tiene del dataset son string o cadenas de texto que el modelo no va a poder interpretar por si solo, se crea la funcion `tratar_categoricas(data)` para que a cada columna que tenga valores de tipo objeto se le reemplace cada categoria por un valor entero unico.
Por ejemplo en la columna `GENRE` se remplazaria ``M`` por 0 y ``F`` por 1

In [35]:
def tratar_categoricas(data):

    #Codifica todas las variables categóricas del DataFrame usando codificación por etiquetas.
    #Cada categoría única se reemplaza por un número entero único.

    columnas_obj = [col for col in data.columns if data[col].dtype == "object"]
    for col in columnas_obj:
        data[col], _ = pd.factorize(data[col])
    return data

### Agregar datos numéricos y categóricos por cliente
La funcion `agregar_por_cliente(data, id_col, etiqueta=None)` se crea con el objetivo de separar las columnas en 2 tipos:
* Categoricas
* Numericas

Esto con el objetivo de poder extraer los estadisticos relevantes de estos 2 tipos de datos.

In [36]:
def agregar_por_cliente(data, id_col, etiqueta=None):
    #Se separan las columnas en categóricas y numéricas. Las categóricas son aquellas con tipo de dato object, y el resto son numéricas.
    categoricas = [col for col in data.columns if data[col].dtype == "object"]
    datos_numericos = data.drop(columns=categoricas)
    datos_categoricos = data[[id_col] + categoricas]

    print(f"- Variables categóricas: {len(categoricas)}")
    print(f"- Variables numéricas: {datos_numericos.shape[1] - 1}")

    # Para las columnas numéricas, se agrupan por el id_col y se calculan varias estadísticas: 
    # la media (mean), la desviación estándar (std), el valor mínimo (min), y el valor máximo (max)
    if datos_numericos.shape[1] > 1:
        numericos_agg = datos_numericos.groupby(id_col).agg(["mean", "std", "min", "max"])
        numericos_agg.columns = [f"{col[0]}_{col[1]}" for col in numericos_agg.columns]
        numericos_agg = numericos_agg.sort_index()
    else:
        numericos_agg = pd.DataFrame()

    # Para las columnas categóricas, también se agrupan por el id_col, y se calculan dos estadísticas:
    # la moda y el numero de valores unicos (cuantas categorias tiene una columna)
    if categoricas:
        categoricos_agg = datos_categoricos.groupby(id_col).agg({
            col: [lambda x: scipy.stats.mode(x)[0][0], lambda x: x.nunique()]
            for col in categoricas
        })
        categoricos_agg.columns = [f"{col[0]}_{col[1]}" for col in categoricos_agg.columns]
        categoricos_agg = categoricos_agg.sort_index()
    else:
        categoricos_agg = pd.DataFrame()

    # Despues de calcular los estadisticos mas relevantes se unen en un solo Dataframe
    if not numericos_agg.empty and not categoricos_agg.empty:
        resultado = pd.concat([numericos_agg, categoricos_agg], axis=1)
    elif not numericos_agg.empty:
        resultado = numericos_agg
    else:
        resultado = categoricos_agg

    if etiqueta:
        resultado.columns = [etiqueta + "_" + col for col in resultado.columns]

    return resultado

# Preprocesamiento de los datos

### Contar valores nulos
La función `contar_nulos(data)` permite encontrar valores faltantes en un conjunto de datos. Su objetivo es identificar qué columnas tienen datos incompletos, cuántos valores faltan y qué porcentaje representan respecto al total de registros.

Primero, calcula la cantidad total de valores nulos en cada columna utilizando `data.isnull().sum()`, que cuenta los `True` para cada celda nula. Luego, ordena estas cantidades de forma descendente para priorizar las columnas más problemáticas.

A continuación, calcula el porcentaje de nulos dividiendo cada total de nulos por el número total de filas del `DataFrame`, y multiplica por 100 para expresarlo en porcentaje. También se ordenan los resultados de mayor a menor.

Ambas series (total y porcentaje) se combinan en un solo `DataFrame` usando `pd.concat.

Por último, se filtran las columnas con valores nulos, devolviendo únicamente aquellas que presentan al menos un valor faltante. Esto permite enfocarse exclusivamente en las variables que requieren algún tipo de tratamiento durante la limpieza de datos.

In [37]:

def contar_nulos(data):
    total = data.isnull().sum().sort_values(ascending=False)
    porcentaje = (data.isnull().sum() / len(data) * 100).sort_values(ascending=False)
    tabla = pd.concat([total, porcentaje], axis=1, keys=["Total de nulos", "Porcentaje"])
    return tabla[tabla["Total de nulos"] > 0]

### Crear indicadores flags de nulos
``crear_flags_nulos(data, columnas=None)`` tiene como propósito generar nuevas variables binarias (0 o 1) que indiquen si un valor estaba originalmente ausente. Esto con el fin de que la ausencia de valores se vuelva una informacion relevante para el modelo.
Por ejemplo, si los clientes con ingresos faltantes tienden a incumplir, el simple hecho de no declarar el ingreso se vuelve información sumamente relevante.



In [38]:

def crear_flags_nulos(data, columnas=None):
    if columnas is None:
        columnas = data.columns
    for col in columnas:
        if data[col].isnull().any():
            data["ES_NULO_" + str(col)] = data[col].isnull().astype(int)
    return data


### Calcular tasas de aprobación y rechazo
Despues de realizar una busqueda sobre modelos de riesgo crediticio se encontro que los modelos de crédito deben de considerar el historial de comportamiento previo como uno de los predictores más potentes.

Esto se puede encontrar en Basel Committee on Banking Supervision — Credit Risk Modelling: Current Practices and Applications (https://www.bis.org/publ/bcbs49.htm) donde se menciona expresamente el uso de variables como el comportamiento pasado del cliente como feature principal o en Anderson, R. (2007). The Credit Scoring Toolkit: Theory and Practice for Retail Credit Risk Management (mirar capitulo 9) donde se menciona que variables como número de rechazos previos, historial de solicitudes y frecuencia de aceptación son claves para desarrollar un analisis completo.

Se procede a construir la funcion `calcular_ratio_aprobacion(data, lags=[1, 2, 5])` que tiene como objetivo calcular, para cada cliente ``(SK_ID_CURR)``, el porcentaje de solicitudes aprobadas y rechazadas en sus últimas t solicitudes de crédito, donde t toma los valores definidos en la lista lags (para este caso elegi los valores: [1, 2, 5]).

se crearan las columnas "RATIO_APROBADO_{t}" donde RATIO_APROBADO_1 es si la ultima solicitud fue aprobada y RATIO_APROBADO_2 es cuántas de las 2 más recientes fueron rechazadas, en promedio. Exactamente lo mismo para el ratio de rechazo

In [39]:

def calcular_ratio_aprobacion(data, lags=[1, 2, 5]):
    #Se extraen solo las columnas necesarias
    df = data[["SK_ID_CURR", "SK_ID_PREV", "DAYS_DECISION", "NAME_CONTRACT_STATUS"]].copy()
    #La columna DAYS_DECISION representa la antigüedad de la solicitud (en días negativos). 
    #Se invierte el signo para que los valores más recientes queden al final, y se ordena por cliente y fecha.
    df["DAYS_DECISION"] = -df["DAYS_DECISION"]
    df = df.sort_values(["SK_ID_CURR", "DAYS_DECISION"])
    #Se aplican dummies sobre la columna NAME_CONTRACT_STATUS para generar columnas binarias como: 1 aprobado o 0 rechazado
    df = pd.get_dummies(df)

    for t in lags:
        tmp = df.groupby("SK_ID_CURR")["NAME_CONTRACT_STATUS_Approved"].head(t).groupby("SK_ID_CURR").mean().reset_index()
        tmp.columns = ["SK_ID_CURR", f"RATIO_APROBADO_{t}"]
        data = data.merge(tmp, on="SK_ID_CURR", how="left")

        tmp = df.groupby("SK_ID_CURR")["NAME_CONTRACT_STATUS_Refused"].head(t).groupby("SK_ID_CURR").mean().reset_index()
        tmp.columns = ["SK_ID_CURR", f"RATIO_RECHAZADO_{t}"]
        data = data.merge(tmp, on="SK_ID_CURR", how="left")

    return data


### Determinar correlacion entre las variables
Una tecnica comun en el preprocesamiento de datos es la reduccion de caracteristicas, esta me dice que si tengo 2 vaiables que tienen una correlacion fuerte (si mi correlacion es igual a 1 entonces son directamente proporcionales y si es -1 son inversamente proporcionales) entonces es redundante tenerlas y puede llegar a dificultar la deteccion de un patron ya que lo que me dice una variable se puede interpretar de la otra. Con esto en mente se procede a calcular la correlacion entre las variables y a eliminar las que estan fuertemente correlacionadas entre si.

In [40]:
def eliminar_variables_correlacionadas(df, umbral=0.9):
    """
    Elimina variables del DataFrame que tienen una correlación fuerte es decir mayor a 0.9
    """
    corr_matrix = df.corr(numeric_only=True).abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Encuentra columnas con correlación alta
    columnas_a_eliminar = [column for column in upper.columns if any(upper[column] > umbral)]
    
    print(f"Se eliminarán {len(columnas_a_eliminar)} columnas por alta correlación.")
    
    df_sin_correlacion = df.drop(columns=columnas_a_eliminar)
    return df_sin_correlacion, columnas_a_eliminar


### Detección y manejo de valores atipicos
Se debe determinar los valores que se salen del comportamiento normal de la variable, para ello se procedera a ver los valores que son superiores al bigote superior (el bigote superior se calcula como el cuartil 3 + 1.5 * el rango intercuartilico, el rango intercuatilico es la diferencia entre el cuartil 3 y el cuartil 1), y el bigote inferior se calcula como Q1-1.5 * el rango intercuartilico.

Si los valores atipicos representan una gran parte de la poblacion entonces la muestra no tiene una comportamiento normal, para esto se determina si los valores atipicos son menores al 7% (este valor no esta respaldado en la literatura, es una asuncion propia) y en caso de que sea menores al 7% se pueden eliminar sin alterar significativamente el resultado

In [41]:
def eliminar_outliers(df, umbral=0.07):
    """
    Elimina los outliers por variable numérica si representan menos del 7% del total de muestras.
    """
    df_limpio = df.copy()
    columnas_numericas = df.select_dtypes(include=[np.number]).columns
    total_filas = len(df)
    outliers_eliminados = []

    for col in columnas_numericas:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        limite_inferior = q1 - 1.5 * iqr
        limite_superior = q3 + 1.5 * iqr

        outliers = df[(df[col] < limite_inferior) | (df[col] > limite_superior)]
        porcentaje_outliers = len(outliers) / total_filas

        if porcentaje_outliers < umbral:
            df_limpio = df_limpio[(df_limpio[col] >= limite_inferior) & (df_limpio[col] <= limite_superior)]
            outliers_eliminados.append(col)
    
    print(f"Outliers eliminados en {len(outliers_eliminados)} columnas: {outliers_eliminados}")
    return df_limpio


### Importar la data

In [42]:
train = pd.read_csv("files/application_train.csv")
test  = pd.read_csv("files/application_test.csv")
previous_application  = pd.read_csv("files/previous_application.csv")
installments  = pd.read_csv("files/installments_payments.csv")

In [43]:
# Extraer la variable objetivo
y = train.select(["SK_ID_CURR", "TARGET"])
train = train.drop("TARGET")
y


SK_ID_CURR,TARGET
i64,i64
100002,1
100003,0
100004,0
100006,0
100007,0
…,…
456251,0
456252,0
456253,0


In [44]:
df_combinado = pd.concat([train, test])
del train, test
df_combinado

SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,…,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
i64,str,str,str,str,i64,f64,f64,f64,f64,str,str,str,str,str,f64,i64,i64,f64,i64,f64,i64,i64,i64,i64,i64,i64,str,f64,i64,i64,str,i64,i64,i64,i64,i64,…,f64,str,str,f64,str,str,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64
100002,"""Cash loans""","""M""","""N""","""Y""",0,202500.0,406597.5,24700.5,351000.0,"""Unaccompanied""","""Working""","""Secondary / secondary special""","""Single / not married""","""House / apartment""",0.018801,-9461,-637,-3648.0,-2120,null,1,1,0,1,1,0,"""Laborers""",1.0,2,2,"""WEDNESDAY""",10,0,0,0,0,…,0.0,"""reg oper account""","""block of flats""",0.0149,"""Stone, brick""","""No""",2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,"""Cash loans""","""F""","""N""","""N""",0,270000.0,1293502.5,35698.5,1.1295e6,"""Family""","""State servant""","""Higher education""","""Married""","""House / apartment""",0.003541,-16765,-1188,-1186.0,-291,null,1,1,0,1,1,0,"""Core staff""",2.0,1,1,"""MONDAY""",11,0,0,0,0,…,0.01,"""reg oper account""","""block of flats""",0.0714,"""Block""","""No""",1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,"""Revolving loans""","""M""","""Y""","""Y""",0,67500.0,135000.0,6750.0,135000.0,"""Unaccompanied""","""Working""","""Secondary / secondary special""","""Single / not married""","""House / apartment""",0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,"""Laborers""",1.0,2,2,"""MONDAY""",9,0,0,0,0,…,null,null,null,null,null,null,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100006,"""Cash loans""","""F""","""N""","""Y""",0,135000.0,312682.5,29686.5,297000.0,"""Unaccompanied""","""Working""","""Secondary / secondary special""","""Civil marriage""","""House / apartment""",0.008019,-19005,-3039,-9833.0,-2437,null,1,1,0,1,0,0,"""Laborers""",2.0,2,2,"""WEDNESDAY""",17,0,0,0,0,…,null,null,null,null,null,null,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,null,null,null
100007,"""Cash loans""","""M""","""N""","""Y""",0,121500.0,513000.0,21865.5,513000.0,"""Unaccompanied""","""Working""","""Secondary / secondary special""","""Single / not married""","""House / apartment""",0.028663,-19932,-3038,-4311.0,-3458,null,1,1,0,1,0,0,"""Core staff""",1.0,2,2,"""THURSDAY""",11,0,0,0,0,…,null,null,null,null,null,null,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

### Columnas que pueden resstructurarse de una mejor manera
Hay varios campos que no me dan información significativa de manera individual pero se puede generar un nuevo campo calculado que resuma varias columnas, por ejemplo son 20 columnas que dan informacion sobre si el cliente entrego un documento X, seria mas practico resumir esas 20 columnas en una sola para evaluar cuantos de esos documentos se entregaron con respecto al total de documentos. Ademas podria sacar otra columna como el ingreso por hogar al dividir los ingresos que tiene una persona por el numero de personas que viven en su hogar "CNT_FAM_MEMBERS" y de esta forma elimino "CNT_FAM_MEMBERS".

In [ ]:
df_combinado = df_combinado.with_columns([
    (pd.col("AMT_INCOME_TOTAL") / pd.col("CNT_FAM_MEMBERS")).alias("INGRESO_POR_PERSONA")
]).drop(["CNT_FAM_MEMBERS"])


In [54]:
docs = [
    "FLAG_DOCUMENT_2",  "FLAG_DOCUMENT_3",  "FLAG_DOCUMENT_4",  "FLAG_DOCUMENT_5",  "FLAG_DOCUMENT_6",
    "FLAG_DOCUMENT_7",  "FLAG_DOCUMENT_8",  "FLAG_DOCUMENT_9",  "FLAG_DOCUMENT_10", "FLAG_DOCUMENT_11",
    "FLAG_DOCUMENT_12", "FLAG_DOCUMENT_13", "FLAG_DOCUMENT_14", "FLAG_DOCUMENT_15", "FLAG_DOCUMENT_16",
    "FLAG_DOCUMENT_17", "FLAG_DOCUMENT_18", "FLAG_DOCUMENT_19", "FLAG_DOCUMENT_20", "FLAG_DOCUMENT_21"
]
#Se suman las columnas que tienen un 1 (es decir que entregaron este documento) y se divide por el numero maximo de documentos a entregar (20)
#Con esto se pueden resumir 20 columnas en una sola

df_combinado = df_combinado.with_columns(
    (sum([pd.col(col) for col in docs]) / 20).alias("PORCENTAJE_DOCUMENTACION_ENTREGADA")
).drop(docs)


In [56]:
df_combinado["PORCENTAJE_DOCUMENTACION_ENTREGADA"].max()

0.2

Se eliminan las columnas que desde el analisis preliminar se determina que no brindan informacion relevante para el analisis.
Si se desea mas detalle de por que se descartaron leer el archivo README.md

In [ ]:
columnas_no_relevantes=['APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 
         'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI',
         'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI','YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI',
         'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'COMMONAREA_MODE','ELEVATORS_MODE', 'ENTRANCES_MODE', 
         'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 
         'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'TOTALAREA_MODE',  'YEARS_BEGINEXPLUATATION_MODE','NAME_TYPE_SUITE', 
         'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 
         'FLAG_PHONE', 'FLAG_EMP_PHONE', "DAY_APPR_PROCESS_START"]
df_combinado = df_combinado.drop(columnas_no_relevantes)

In [ ]:
# convertir a escala logaritmica
var_logaritmicas = ["AMT_CREDIT", "AMT_GOODS_PRICE", "AMT_ANNUITY","AMT_INCOME_TOTAL"]
df_combinado = crear_logaritmos(df_combinado, var_logaritmicas, replace = True)

# convertir dias negativos a meses
variables_dias_negativos = ["DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_LAST_PHONE_CHANGE"]
df_combinado = convertir_dias(df_combinado, variables_dias_negativos, t = 30, rounding = True, replace = True)